In [1]:
# dataset
!wget https://dl.fbaipublicfiles.com/glue/data/WNLI.zip


--2023-01-30 11:12:07--  https://dl.fbaipublicfiles.com/glue/data/WNLI.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28999 (28K) [application/zip]
Saving to: ‘WNLI.zip’

WNLI.zip            100%[===================>]  28.32K  --.-KB/s    in 0.007s  

2023-01-30 11:12:08 (3.89 MB/s) - ‘WNLI.zip’ saved [28999/28999]



In [2]:
!unzip WNLI.zip


Archive:  WNLI.zip
   creating: WNLI/
  inflating: WNLI/dev.tsv            
  inflating: WNLI/test.tsv           
  inflating: WNLI/train.tsv          


In [3]:
import pandas as pd

df_train = pd.read_csv("WNLI/train.tsv", sep='\t')
df_train.head()

,index,sentence1,sentence2,label
0,0,I stuck a pin through a carrot. When I pulled ...,The carrot had a hole.,1
1,1,John couldn't see the stage with Billy in fron...,John is so short.,1
2,2,The police arrested all of the gang members. T...,The police were trying to stop the drug trade ...,1
3,3,Steve follows Fred's example in everything. He...,Steve influences him hugely.,0
4,4,"When Tatyana reached the cabin, her mother was...","mother was careful not to disturb her, undress...",0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635 entries, 0 to 634
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      635 non-null    int64 
 1   sentence1  635 non-null    object
 2   sentence2  635 non-null    object
 3   label      635 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 20.0+ KB


In [5]:
df_dev = pd.read_csv("WNLI/dev.tsv", sep='\t')
df_dev.head()

,index,sentence1,sentence2,label
0,0,The drain is clogged with hair. It has to be c...,The hair has to be cleaned.,0
1,1,Jane knocked on Susan's door but she did not a...,Susan did not answer.,1
2,2,"Beth didn't get angry with Sally, who had cut ...",Sally stopped and counted to ten.,0
3,3,No one joins Facebook to be sad and lonely. Bu...,That's exactly how Facebook makes us feel.,1
4,4,The man couldn't lift his son because he was s...,The son was so heavy.,1


In [6]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      71 non-null     int64 
 1   sentence1  71 non-null     object
 2   sentence2  71 non-null     object
 3   label      71 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.3+ KB


In [7]:
df_test = pd.read_csv("WNLI/test.tsv", sep='\t')
df_test.head()

,index,sentence1,sentence2
0,0,Maude and Dora had seen the trains rushing acr...,Horses ran away when Maude and Dora came in si...
1,1,Maude and Dora had seen the trains rushing acr...,Horses ran away when the trains came in sight.
2,2,Maude and Dora had seen the trains rushing acr...,Horses ran away when the puffs came in sight.
3,3,Maude and Dora had seen the trains rushing acr...,Horses ran away when the roars came in sight.
4,4,Maude and Dora had seen the trains rushing acr...,Horses ran away when the whistles came in sight.


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      146 non-null    int64 
 1   sentence1  146 non-null    object
 2   sentence2  146 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.5+ KB


In [9]:
def make_prompt(sentence_1, sentence_2):
  return f'Having the sentence "{sentence_1}" tell me if the following sentence "{sentence_2}" is true or false? Answer a number "0" if false or "1" if true.'

In [10]:
df_test['label'] = "unknown"
df_test['split'] = 'test'

df_dev['split'] = 'dev'
df_train['split'] = 'train'

In [11]:
df_all = pd.concat([df_train, df_dev, df_test])
df_all.head()

,index,sentence1,sentence2,label,split
0,0,I stuck a pin through a carrot. When I pulled ...,The carrot had a hole.,1,train
1,1,John couldn't see the stage with Billy in fron...,John is so short.,1,train
2,2,The police arrested all of the gang members. T...,The police were trying to stop the drug trade ...,1,train
3,3,Steve follows Fred's example in everything. He...,Steve influences him hugely.,0,train
4,4,"When Tatyana reached the cabin, her mother was...","mother was careful not to disturb her, undress...",0,train


In [15]:
sentences_1 = df_all.sentence1.values.tolist()
sentences_2 = df_all.sentence2.values.tolist()
prompts = []

for sentence_1, sentence_2 in zip(sentences_1, sentences_2):
  prompt = make_prompt(sentence_1, sentence_2)
  prompts.append(prompt)

In [16]:
import numpy as np

df_all['prompt'] = prompts
df_all['chatgpt_answer'] = np.nan

In [17]:
df_all.head()

,index,sentence1,sentence2,label,split,prompt,chatgpt_answer
0,0,I stuck a pin through a carrot. When I pulled ...,The carrot had a hole.,1,train,"Having the sentence ""I stuck a pin through a c...",NaN
1,1,John couldn't see the stage with Billy in fron...,John is so short.,1,train,"Having the sentence ""John couldn't see the sta...",NaN
2,2,The police arrested all of the gang members. T...,The police were trying to stop the drug trade ...,1,train,"Having the sentence ""The police arrested all o...",NaN
3,3,Steve follows Fred's example in everything. He...,Steve influences him hugely.,0,train,"Having the sentence ""Steve follows Fred's exam...",NaN
4,4,"When Tatyana reached the cabin, her mother was...","mother was careful not to disturb her, undress...",0,train,"Having the sentence ""When Tatyana reached the ...",NaN


In [18]:
df_all.to_csv("winograd_nli_prompts.tsv", sep="\t")

In [19]:
df_all_explain = df_all.copy()

df_all_explain['prompt'] = df_all_explain['prompt'].apply(lambda x: x + ' Explain your answer.')

df_all_explain.head()

,index,sentence1,sentence2,label,split,prompt,chatgpt_answer
0,0,I stuck a pin through a carrot. When I pulled ...,The carrot had a hole.,1,train,"Having the sentence ""I stuck a pin through a c...",NaN
1,1,John couldn't see the stage with Billy in fron...,John is so short.,1,train,"Having the sentence ""John couldn't see the sta...",NaN
2,2,The police arrested all of the gang members. T...,The police were trying to stop the drug trade ...,1,train,"Having the sentence ""The police arrested all o...",NaN
3,3,Steve follows Fred's example in everything. He...,Steve influences him hugely.,0,train,"Having the sentence ""Steve follows Fred's exam...",NaN
4,4,"When Tatyana reached the cabin, her mother was...","mother was careful not to disturb her, undress...",0,train,"Having the sentence ""When Tatyana reached the ...",NaN


In [20]:
df_all_explain.to_csv("winograd_nli_prompts_explain.tsv", sep="\t")

In [21]:
df_all_explain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 852 entries, 0 to 145
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           852 non-null    int64  
 1   sentence1       852 non-null    object 
 2   sentence2       852 non-null    object 
 3   label           852 non-null    object 
 4   split           852 non-null    object 
 5   prompt          852 non-null    object 
 6   chatgpt_answer  0 non-null      float64
dtypes: float64(1), int64(1), object(5)
memory usage: 53.2+ KB
